# Lekcja 5: Bazy danych SQLite3 w Pythonie

SQLite to lekka, wbudowana baza danych, która nie wymaga oddzielnego serwera. Jest idealna do nauki SQL oraz małych i średnich aplikacji.

## 1. Podstawy - Połączenie z bazą danych

Pierwszym krokiem jest zaimportowanie modułu `sqlite3` i utworzenie połączenia z bazą danych.

In [ ]:
import sqlite3
from sqlite3 import connect

# Utworzenie połączenia z bazą danych (plik zostanie utworzony jeśli nie istnieje)
connection = sqlite3.connect('w05_sqlite_example_database.db')

# Utworzenie kursora do wykonywania zapytań SQL
cursor = connection.cursor()

print("Połączenie z bazą danych zostało utworzone!")

### Baza danych w pamięci

Możemy również utworzyć bazę danych w pamięci RAM (dane znikną po zamknięciu programu):

In [ ]:
# Baza danych w pamięci - idealna do testów
conn_ram = sqlite3.connect(':memory:')
cursor = conn_ram.cursor()

print("Baza danych w pamięci została utworzona!")

## 2. Tworzenie tabeli

Tabela to struktura przechowująca dane w postaci wierszy i kolumn.

In [ ]:
# Tworzenie prostej tabeli użytkowników
cursor.execute('''
    CREATE TABLE IF NOT EXISTS uzytkownicy (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        imie TEXT NOT NULL,
        nazwisko TEXT NOT NULL,
        wiek INTEGER,
        email TEXT UNIQUE
    )
''')

# Zapisanie zmian
connection.commit()

print("Tabela 'uzytkownicy' została utworzona!")

## 3. Dodawanie danych (INSERT)

Wstawianie pojedynczych rekordów do tabeli.

In [ ]:
# Wstawianie pojedynczego rekordu
cursor.execute('''
    INSERT INTO uzytkownicy (imie, nazwisko, wiek, email)
    VALUES ('Jan', 'Kowalski', 25, 'jan.kowalski@example.com')
''')

connection.commit()
print("Dodano użytkownika Jana Kowalskiego")

### Bezpieczne wstawianie danych (parametryzowane zapytania)

Używaj zawsze parametryzowanych zapytań, aby uniknąć SQL Injection!

In [ ]:
# Bezpieczne wstawianie danych
dane_uzytkownika = ('Anna', 'Nowak', 30, 'anna.nowak@example.com')

cursor.execute('''
    INSERT INTO uzytkownicy (imie, nazwisko, wiek, email)
    VALUES (?, ?, ?, ?)
''', dane_uzytkownika)

connection.commit()
print(f"Dodano użytkownika: {dane_uzytkownika[0]} {dane_uzytkownika[1]}")

### Wstawianie wielu rekordów naraz

In [ ]:
# Wstawianie wielu użytkowników jednocześnie
wielu_uzytkownikow = [
    ('Piotr', 'Wiśniewski', 28, 'piotr.wisniewski@example.com'),
    ('Ewa', 'Kowalczyk', 35, 'ewa.kowalczyk@example.com'),
    ('Tomasz', 'Lewandowski', 42, 'tomasz.lewandowski@example.com')
]

cursor.executemany('''
    INSERT INTO uzytkownicy (imie, nazwisko, wiek, email)
    VALUES (?, ?, ?, ?)
''', wielu_uzytkownikow)

connection.commit()
print(f"Dodano {len(wielu_uzytkownikow)} użytkowników")

## 4. Odczytywanie danych (SELECT)

Pobieranie danych z tabeli.

In [ ]:
# Pobranie wszystkich użytkowników
cursor.execute('SELECT * FROM uzytkownicy')
wszyscy = cursor.fetchall()

print("Wszyscy użytkownicy:")
for uzytkownik in wszyscy:
    print(uzytkownik)

### Pobieranie wybranych kolumn

In [ ]:
# Pobranie tylko imion i nazwisk
cursor.execute('SELECT imie, nazwisko FROM uzytkownicy')
imiona_nazwiska = cursor.fetchall()

print("Imiona i nazwiska:")
for imie, nazwisko in imiona_nazwiska:
    print(f"{imie} {nazwisko}")

### Pobieranie jednego rekordu

In [ ]:
# Pobranie pierwszego użytkownika
cursor.execute('SELECT * FROM uzytkownicy LIMIT 1')
pierwszy = cursor.fetchone()

print("Pierwszy użytkownik:", pierwszy)

## 5. Filtrowanie danych (WHERE)

Wyszukiwanie użytkowników spełniających określone kryteria.

In [ ]:
# Wyszukiwanie użytkowników starszych niż 30 lat
cursor.execute('SELECT * FROM uzytkownicy WHERE wiek > ?', (30,))
starsi = cursor.fetchall()

print("Użytkownicy starsi niż 30 lat:")
for uzytkownik in starsi:
    print(f"{uzytkownik[1]} {uzytkownik[2]}, wiek: {uzytkownik[3]}")

In [ ]:
# Wyszukiwanie po imieniu (LIKE)
cursor.execute("SELECT * FROM uzytkownicy WHERE imie LIKE 'A%'")
na_a = cursor.fetchall()

print("Użytkownicy których imię zaczyna się na 'A':")
for uzytkownik in na_a:
    print(f"{uzytkownik[1]} {uzytkownik[2]}")

## 6. Aktualizacja danych (UPDATE)

Modyfikowanie istniejących rekordów.

In [ ]:
# Aktualizacja wieku użytkownika
cursor.execute('''
    UPDATE uzytkownicy
    SET wiek = ?
    WHERE email = ?
''', (26, 'jan.kowalski@example.com'))

connection.commit()
print(f"Zaktualizowano {cursor.rowcount} rekord(ów)")

In [ ]:
# Sprawdzenie zmian
cursor.execute("SELECT * FROM uzytkownicy WHERE email = ?", ('jan.kowalski@example.com',))
print("Zaktualizowany użytkownik:", cursor.fetchone())

## 7. Usuwanie danych (DELETE)

Usuwanie rekordów z tabeli.

In [ ]:
# Dodajmy użytkownika do usunięcia
cursor.execute('''
    INSERT INTO uzytkownicy (imie, nazwisko, wiek, email)
    VALUES (?, ?, ?, ?)
''', ('Test', 'Testowy', 99, 'test@example.com'))
connection.commit()

# Usunięcie użytkownika
cursor.execute('DELETE FROM uzytkownicy WHERE email = ?', ('test@example.com',))
connection.commit()

print(f"Usunięto {cursor.rowcount} rekord(ów)")

## 8. Zaawansowane - Sortowanie (ORDER BY)

In [ ]:
# Sortowanie według wieku (rosnąco)
cursor.execute('SELECT imie, nazwisko, wiek FROM uzytkownicy ORDER BY wiek ASC')
print("Użytkownicy posortowani według wieku (rosnąco):")
for uzytkownik in cursor.fetchall():
    print(f"{uzytkownik[0]} {uzytkownik[1]}: {uzytkownik[2]} lat")

In [ ]:
# Sortowanie według nazwiska (malejąco)
cursor.execute('SELECT imie, nazwisko FROM uzytkownicy ORDER BY nazwisko DESC')
print("\nUżytkownicy posortowani według nazwiska (malejąco):")
for uzytkownik in cursor.fetchall():
    print(f"{uzytkownik[0]} {uzytkownik[1]}")

## 9. Zaawansowane - Grupowanie (GROUP BY) i funkcje agregujące

In [ ]:
# Średni wiek użytkowników
cursor.execute('SELECT AVG(wiek) as sredni_wiek FROM uzytkownicy')
srednia = cursor.fetchone()[0]
print(f"Średni wiek użytkowników: {srednia:.2f} lat")

In [ ]:
# Liczba użytkowników
cursor.execute('SELECT COUNT(*) as liczba FROM uzytkownicy')
liczba = cursor.fetchone()[0]
print(f"Liczba użytkowników w bazie: {liczba}")

In [ ]:
# Najstarszy i najmłodszy użytkownik
cursor.execute('SELECT MAX(wiek) as najstarszy, MIN(wiek) as najmlodszy FROM uzytkownicy')
wynik = cursor.fetchone()
print(f"Najstarszy użytkownik: {wynik[0]} lat")
print(f"Najmłodszy użytkownik: {wynik[1]} lat")

## 10. Zaawansowane - Relacje między tabelami (JOIN)

Stwórzmy dodatkową tabelę z zamówieniami użytkowników.

In [ ]:
# Tworzenie tabeli zamówień
cursor.execute('''
    CREATE TABLE IF NOT EXISTS zamowienia (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        uzytkownik_id INTEGER,
        produkt TEXT NOT NULL,
        cena REAL NOT NULL,
        data TEXT NOT NULL,
        FOREIGN KEY (uzytkownik_id) REFERENCES uzytkownicy(id)
    )
''')

connection.commit()
print("Tabela 'zamowienia' została utworzona!")

In [ ]:
# Dodanie przykładowych zamówień
zamowienia = [
    (1, 'Laptop', 3500.00, '2024-01-15'),
    (1, 'Mysz', 50.00, '2024-01-15'),
    (2, 'Klawiatura', 200.00, '2024-01-16'),
    (3, 'Monitor', 1200.00, '2024-01-17'),
    (3, 'Głośniki', 300.00, '2024-01-17')
]

cursor.executemany('''
    INSERT INTO zamowienia (uzytkownik_id, produkt, cena, data)
    VALUES (?, ?, ?, ?)
''', zamowienia)

connection.commit()
print(f"Dodano {len(zamowienia)} zamówień")

In [ ]:
# INNER JOIN - połączenie użytkowników z ich zamówieniami
cursor.execute('''
    SELECT u.imie, u.nazwisko, z.produkt, z.cena, z.data
    FROM uzytkownicy u
    INNER JOIN zamowienia z ON u.id = z.uzytkownik_id
    ORDER BY z.data
''')

print("Użytkownicy i ich zamówienia:")
for row in cursor.fetchall():
    print(f"{row[0]} {row[1]} zamówił(a) {row[2]} za {row[3]} zł w dniu {row[4]}")

In [ ]:
# Suma zamówień dla każdego użytkownika
cursor.execute('''
    SELECT u.imie, u.nazwisko, SUM(z.cena) as suma_zamowien, COUNT(z.id) as liczba_zamowien
    FROM uzytkownicy u
    INNER JOIN zamowienia z ON u.id = z.uzytkownik_id
    GROUP BY u.id
    ORDER BY suma_zamowien DESC
''')

print("\nPodsumowanie zamówień:")
for row in cursor.fetchall():
    print(f"{row[0]} {row[1]}: {row[2]} zł ({row[3]} zamówień)")

## 11. Zaawansowane - Transakcje

Transakcje pozwalają na grupowanie operacji - albo wszystkie się wykonają, albo żadna.

In [ ]:
try:
    # Rozpoczęcie transakcji (automatycznie po każdym execute)
    cursor.execute('''
        INSERT INTO uzytkownicy (imie, nazwisko, wiek, email)
        VALUES (?, ?, ?, ?)
    ''', ('Marek', 'Testowy', 33, 'marek.test@example.com'))
    
    # Pobranie ID nowo dodanego użytkownika
    nowy_id = cursor.lastrowid
    
    cursor.execute('''
        INSERT INTO zamowienia (uzytkownik_id, produkt, cena, data)
        VALUES (?, ?, ?, ?)
    ''', (nowy_id, 'Tablet', 1500.00, '2024-01-20'))
    
    # Zatwierdzenie transakcji
    connection.commit()
    print("Transakcja zakończona pomyślnie!")
    
except sqlite3.Error as e:
    # Wycofanie zmian w przypadku błędu
    connection.rollback()
    print(f"Błąd: {e}. Transakcja została wycofana.")

## 12. Zaawansowane - Context Manager (with)

Używanie `with` zapewnia automatyczne zamknięcie połączenia.

In [ ]:
# Użycie context managera
with sqlite3.connect('moja_baza.db') as conn:
    cur = conn.cursor()
    cur.execute('SELECT COUNT(*) FROM uzytkownicy')
    liczba = cur.fetchone()[0]
    print(f"Liczba użytkowników: {liczba}")
    # Połączenie zostanie automatycznie zamknięte

## 13. Zaawansowane - Row Factory (dostęp do kolumn po nazwie)

In [ ]:
# Ustawienie row_factory aby móc odwoływać się do kolumn po nazwie
connection.row_factory = sqlite3.Row
cursor = connection.cursor()

cursor.execute('SELECT * FROM uzytkownicy LIMIT 3')

print("Użytkownicy (dostęp po nazwach kolumn):")
for row in cursor.fetchall():
    print(f"ID: {row['id']}, Imię: {row['imie']}, Nazwisko: {row['nazwisko']}, Wiek: {row['wiek']}")

## 14. Praktyczny przykład - prosty system zarządzania biblioteką

In [ ]:
# Tworzenie bazy danych biblioteki
biblioteka = sqlite3.connect(':memory:')
bib_kursor = biblioteka.cursor()

# Tabela książek
bib_kursor.execute('''
    CREATE TABLE ksiazki (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        tytul TEXT NOT NULL,
        autor TEXT NOT NULL,
        rok_wydania INTEGER,
        dostepna BOOLEAN DEFAULT 1
    )
''')

# Tabela wypożyczeń
bib_kursor.execute('''
    CREATE TABLE wypozyczenia (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        ksiazka_id INTEGER,
        czytelnik TEXT NOT NULL,
        data_wypozyczenia TEXT NOT NULL,
        data_zwrotu TEXT,
        FOREIGN KEY (ksiazka_id) REFERENCES ksiazki(id)
    )
''')

print("Baza danych biblioteki została utworzona!")

In [ ]:
# Dodanie książek do biblioteki
ksiazki = [
    ('Pan Tadeusz', 'Adam Mickiewicz', 1834),
    ('Lalka', 'Bolesław Prus', 1890),
    ('Quo Vadis', 'Henryk Sienkiewicz', 1896),
    ('Chłopi', 'Władysław Reymont', 1904),
    ('Potop', 'Henryk Sienkiewicz', 1886)
]

bib_kursor.executemany('''
    INSERT INTO ksiazki (tytul, autor, rok_wydania)
    VALUES (?, ?, ?)
''', ksiazki)

biblioteka.commit()
print(f"Dodano {len(ksiazki)} książek do biblioteki")

In [ ]:
# Funkcja wypożyczania książki
def wypozycz_ksiazke(ksiazka_id, czytelnik, data):
    # Sprawdzenie czy książka jest dostępna
    bib_kursor.execute('SELECT dostepna FROM ksiazki WHERE id = ?', (ksiazka_id,))
    dostepna = bib_kursor.fetchone()[0]
    
    if dostepna:
        # Wypożyczenie książki
        bib_kursor.execute('''
            INSERT INTO wypozyczenia (ksiazka_id, czytelnik, data_wypozyczenia)
            VALUES (?, ?, ?)
        ''', (ksiazka_id, czytelnik, data))
        
        # Oznaczenie książki jako niedostępnej
        bib_kursor.execute('''
            UPDATE ksiazki SET dostepna = 0 WHERE id = ?
        ''', (ksiazka_id,))
        
        biblioteka.commit()
        print(f"Książka ID {ksiazka_id} została wypożyczona dla {czytelnik}")
    else:
        print("Książka jest już wypożyczona!")

# Wypożyczenie kilku książek
wypozycz_ksiazke(1, 'Jan Kowalski', '2024-01-10')
wypozycz_ksiazke(3, 'Anna Nowak', '2024-01-12')

In [ ]:
# Wyświetlenie dostępnych książek
bib_kursor.execute('''
    SELECT tytul, autor, rok_wydania
    FROM ksiazki
    WHERE dostepna = 1
''')

print("Dostępne książki:")
for ksiazka in bib_kursor.fetchall():
    print(f"- {ksiazka[0]} ({ksiazka[1]}, {ksiazka[2]})")

In [ ]:
# Wyświetlenie aktywnych wypożyczeń
bib_kursor.execute('''
    SELECT k.tytul, w.czytelnik, w.data_wypozyczenia
    FROM wypozyczenia w
    INNER JOIN ksiazki k ON w.ksiazka_id = k.id
    WHERE w.data_zwrotu IS NULL
''')

print("\nAktywne wypożyczenia:")
for wypozyczenie in bib_kursor.fetchall():
    print(f"- {wypozyczenie[0]} wypożyczona przez {wypozyczenie[1]} dnia {wypozyczenie[2]}")

## 15. Zamykanie połączenia

Zawsze pamiętaj o zamknięciu połączenia z bazą danych!

In [ ]:
# Zamknięcie połączenia
connection.close()
biblioteka.close()

print("Połączenia z bazami danych zostały zamknięte.")

## Podsumowanie

W tej lekcji nauczyliśmy się:

1. **Podstawy**: Tworzenie połączenia z bazą danych SQLite
2. **DDL**: Tworzenie tabel (`CREATE TABLE`)
3. **DML - INSERT**: Dodawanie danych do tabel
4. **DML - SELECT**: Odczytywanie danych z tabel
5. **Filtrowanie**: Używanie klauzuli `WHERE` i `LIKE`
6. **DML - UPDATE**: Aktualizacja istniejących danych
7. **DML - DELETE**: Usuwanie danych
8. **Sortowanie**: Używanie `ORDER BY`
9. **Agregacje**: Funkcje `COUNT`, `AVG`, `MAX`, `MIN` i `GROUP BY`
10. **Relacje**: Łączenie tabel za pomocą `JOIN`
11. **Transakcje**: Grupowanie operacji w transakcje
12. **Context Manager**: Bezpieczne zarządzanie połączeniem
13. **Row Factory**: Dostęp do kolumn po nazwie
14. **Praktyka**: Implementacja systemu biblioteki

### Najważniejsze zasady:

- Zawsze używaj **parametryzowanych zapytań** (?) aby uniknąć SQL Injection
- Pamiętaj o **commit()** po operacjach modyfikujących dane
- Używaj **context managera** (`with`) dla bezpiecznego zarządzania połączeniem
- **Zamykaj połączenia** po zakończeniu pracy z bazą danych